# Threshold Sweep + Official PhysioNet 2019 Evaluation

- **Model scripts (e.g., LightGBM, TimesLib)** only write `_WORK` folders containing continuous scores (e.g., `PredictedProbability`).
- **This notebook** is the *single source of truth* for:
  - Threshold sweep on **TRAIN_THRESH** (utility via official evaluator)
  - Selecting best threshold
  - Writing final TimesLib-style folders:
    - `pred_psv_TRAIN_THRESH`
    - `pred_psv_TEST`
  - Official metrics + plots


In [1]:
# ============================================================
# Threshold Sweep + Official PhysioNet 2019 Evaluation
# SAFE + MODEL-AGNOSTIC + Option B
#
# You set ONLY:
#   MODEL_PARENT_DIR
#
# Works for:
# - TimesLib: .../Predictions/TimesLib/<ModelName>/
#   (expects pred_psv_TRAIN_THRESH_WORK and optionally pred_psv_TEST)
# - LightGBM: .../Predictions/LIGHTGBM/
#   (expects pred_psv_TRAIN_THRESH_WORK and pred_psv_TEST_WORK)
#
# Key safety:
# - NEVER overwrites/deletes your existing pred_psv_* folders
# - Writes final thresholded outputs into *_THRESHED folders
# ============================================================

from __future__ import annotations

from pathlib import Path
import json
import shutil
import importlib.util

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ============================
# CONFIG (EDIT ONLY THIS)
# ============================

#MODEL_PARENT_DIR = Path("/teamspace/studios/this_studio/detecting_Sepsis/4_Evaluation/Predictions/TimesLib/Crossformer_HighPreproc_NoFe")
#MODEL_PARENT_DIR = Path("/teamspace/studios/this_studio/detecting_Sepsis/3_Model/Time-Series-Library/results/classification_Sepsis_TFT_HIGH_TemporalFusionTransformer_SepsisPSV_ftM_sl48_ll48_pl0_dm64_nh4_el2_dl1_df256_expand2_dc4_fc1_ebtimeF_dtTrue_HIGH_PREPROC_NO_FE_0")
#MODEL_PARENT_DIR = Path("/teamspace/studios/this_studio/detecting_Sepsis/4_Evaluation/Predictions/LIGHTGBM_No_Preproc")
#MODEL_PARENT_DIR = Path("/teamspace/studios/this_studio/detecting_Sepsis/4_Evaluation/Predictions/TimesLib/TFT_HighPreproc_NoFe")
#MODEL_PARENT_DIR =Path("/teamspace/studios/this_studio/detecting_Sepsis/3_Model/Time-Series-Library/results/classification_Sepsis_TFT_HIGH_TemporalFusionTransformer_SepsisPSV_ftM_sl48_ll48_pl0_dm64_nh4_el2_dl1_df256_expand2_dc4_fc1_ebtimeF_dtTrue_TFT_seq48_bs512_lr1e-4_dm64_df256_do03_p1_pw20_NOsampler_clip1_0")
#MODEL_PARENT_DIR =Path("/teamspace/studios/this_studio/detecting_Sepsis/4_Evaluation/Predictions/qSOFA_aggresive_Alarm_Policy")
MODEL_PARENT_DIR = Path ("/teamspace/studios/this_studio/detecting_Sepsis/3_Model/Time-Series-Library/results/classification_Sepsis_TFT_HIGH_Mean_Impute_TemporalFusionTransformer_SepsisCSV_ftM_sl48_ll48_pl0_dm64_nh4_el2_dl1_df256_expand2_dc4_fc1_ebtimeF_dtTrue_TFT_on_CSV_High_Mean_Impute_0")

# Original Setup 
# Two-stage sweep params (fast + good)
#COARSE_STEP = 0.05
#REFINE_HALF_WIDTH = 0.03
#REFINE_STEP = 0.003

# slower, more accurate setup
COARSE_STEP = 0.01
REFINE_HALF_WIDTH = 0.0015
REFINE_STEP = 0.00015


# ============================
# Helper: locate repo root + label dirs
# ============================
def find_repo_root_from_model_dir(model_parent_dir: Path) -> Path:
    parts = model_parent_dir.resolve().parts
    if "detecting_Sepsis" not in parts:
        raise ValueError(f"MODEL_PARENT_DIR must contain 'detecting_Sepsis' in its path: {model_parent_dir}")
    i = parts.index("detecting_Sepsis")
    return Path(*parts[: i + 1])

def get_label_dirs(repo_root: Path) -> tuple[Path, Path]:
    train_thresh = repo_root / "data" / "raw_PSV" / "PSV_Patients_THRESH"
    test = repo_root / "data" / "raw_PSV" / "PSV_Patients_TEST"
    if not train_thresh.exists():
        raise FileNotFoundError(f"Missing label dir: {train_thresh}")
    if not test.exists():
        print(f"WARNING: Missing TEST label dir: {test} (TEST eval will be skipped)")
    return train_thresh, test


# ============================
# Helper: find prediction dirs (TimesLib + LightGBM compatible)
# ============================
def dir_has_psv(d: Path) -> bool:
    return d.exists() and any(d.glob("*.psv"))

def pick_existing_dir_with_psv(parent: Path, candidates: list[str], required: bool) -> Path | None:
    for c in candidates:
        p = parent / c
        if dir_has_psv(p):
            return p
    if required:
        raise FileNotFoundError(f"No .psv files found in any of: {[(parent / c) for c in candidates]}")
    return None

def get_pred_dirs(model_parent_dir: Path) -> tuple[Path, Path | None]:
    # TRAIN is required
    train_dir = pick_existing_dir_with_psv(
        model_parent_dir,
        ["pred_psv_TRAIN_THRESH_WORK", "pred_psv_TRAIN_THRESH"],
        required=True
    )

    # TEST is optional
    test_dir = pick_existing_dir_with_psv(
        model_parent_dir,
        ["pred_psv_TEST_WORK", "pred_psv_TEST"],
        required=False
    )
    if test_dir is None:
        print("NOTE: No TEST prediction .psv found (pred_psv_TEST_WORK/pred_psv_TEST). Skipping TEST outputs.")

    return train_dir, test_dir


# ============================
# Official evaluator
# ============================
def load_official_evaluator(repo_root: Path):
    eval_script = repo_root / "4_Evaluation" / "evaluate_sepsis_score.py"
    if not eval_script.exists():
        raise FileNotFoundError(f"Official evaluator not found: {eval_script}")
    spec = importlib.util.spec_from_file_location("eval_sepsis", str(eval_script))
    if spec is None or spec.loader is None:
        raise RuntimeError(f"Could not import evaluator from {eval_script}")
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

def official_eval(evaluator_module, label_dir: Path, pred_dir: Path):
    # returns: auroc, auprc, accuracy, f_measure, normalized_observed_utility
    return evaluator_module.evaluate_sepsis_score(str(label_dir), str(pred_dir))


# ============================
# Materialization (SAFE: writes to *_THRESHED only)
# ============================
def detect_score_column(df: pd.DataFrame) -> str:
    # ignore model-provided PredictedLabel; only use score/prob column
    candidates = ["PredictedProbability"]
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(f"No score column found. Columns: {list(df.columns)}")

def ensure_empty_dir(dir_path: Path):
    dir_path.mkdir(parents=True, exist_ok=True)
    for f in dir_path.glob("*.psv"):
        f.unlink()

def write_pred_file_for_official(out_path: Path, prob: np.ndarray, threshold: float):
    pred = (prob >= threshold).astype(int)
    out_df = pd.DataFrame({
        "PredictedProbability": prob.astype(float),
        "PredictedLabel": pred.astype(int),
    })
    out_df.to_csv(out_path, sep="|", index=False)

def materialize_predictions_from_source(source_dir: Path, out_dir: Path, threshold: float):
    """
    Reads .psv from source_dir (probabilities)
    Writes official-format .psv into out_dir (prob + label)
    SAFE because out_dir is distinct from source_dir (we enforce that in naming).
    """
    files = sorted(source_dir.glob("*.psv"))
    if not files:
        raise FileNotFoundError(f"No .psv files found in: {source_dir}")

    ensure_empty_dir(out_dir)

    for f in files:
        df = pd.read_csv(f, sep="|")
        score_col = detect_score_column(df)
        prob = df[score_col].astype(float).to_numpy()
        write_pred_file_for_official(out_dir / f.name, prob, threshold)

def label_dir_has_sepsislabel(label_dir: Path) -> bool:
    files = list(label_dir.glob("*.psv"))
    if not files:
        return False
    df = pd.read_csv(files[0], sep="|", nrows=2)
    return "SepsisLabel" in df.columns


# ============================
# Two-stage sweep (TRAIN_THRESH ONLY)
# ============================
def run_threshold_sweep_two_stage(
    evaluator,
    label_dir_train_thresh: Path,
    pred_source_train: Path,
    tmp_pred_dir: Path,
    coarse_step: float,
    refine_half_width: float,
    refine_step: float,
) -> pd.DataFrame:

    coarse = np.round(np.arange(0.0, 1.0 + 1e-12, coarse_step), 6)

    rows = []
    best_coarse_t = None
    best_coarse_util = -1e18

    print(f"[Stage 1/2] Coarse sweep: step={coarse_step} ({len(coarse)} thresholds)")
    for i, t in enumerate(coarse, 1):
        materialize_predictions_from_source(pred_source_train, tmp_pred_dir, float(t))
        auroc, auprc, acc, f1, util = official_eval(evaluator, label_dir_train_thresh, tmp_pred_dir)
        rows.append((float(t), float(util), float(auroc), float(auprc), float(acc), float(f1)))

        if util > best_coarse_util:
            best_coarse_util = float(util)
            best_coarse_t = float(t)
            print(f"  NEW BEST (coarse) t={best_coarse_t:.4f} util={best_coarse_util:.6f}")

        if i % 10 == 0:
            print(f"  Coarse progress {i}/{len(coarse)} (latest t={t:.4f}, util={float(util):.6f})")

    lo = max(0.0, best_coarse_t - refine_half_width)
    hi = min(1.0, best_coarse_t + refine_half_width)
    refine = np.round(np.arange(lo, hi + 1e-12, refine_step), 6)

    # avoid duplicates
    done = set(np.round(coarse, 6).tolist())
    refine = np.array([t for t in refine if float(t) not in done], dtype=float)

    print(
        f"[Stage 2/2] Refine around {best_coarse_t:.4f}: "
        f"[{lo:.4f}, {hi:.4f}] step={refine_step} ({len(refine)} thresholds)"
    )

    for i, t in enumerate(refine, 1):
        materialize_predictions_from_source(pred_source_train, tmp_pred_dir, float(t))
        auroc, auprc, acc, f1, util = official_eval(evaluator, label_dir_train_thresh, tmp_pred_dir)
        rows.append((float(t), float(util), float(auroc), float(auprc), float(acc), float(f1)))

        if i % 25 == 0:
            print(f"  Refine progress {i}/{len(refine)} (latest t={t:.4f}, util={float(util):.6f})")

    df = pd.DataFrame(rows, columns=["threshold", "utility", "auroc", "auprc", "accuracy", "f1"])
    df = df.sort_values("threshold").reset_index(drop=True)
    return df

def pick_best_threshold(df: pd.DataFrame) -> float:
    best_row = df.sort_values(["utility", "threshold"], ascending=[False, True]).iloc[0]
    return float(best_row["threshold"])

def plot_utility(df: pd.DataFrame, out_png: Path):
    plt.figure()
    plt.plot(df["threshold"].to_numpy(), df["utility"].to_numpy())
    plt.xlabel("Threshold")
    plt.ylabel("Utility (official)")
    plt.title("PhysioNet 2019 Utility vs Threshold")
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()


# ============================
# RUN
# ============================
MODEL_PARENT_DIR = MODEL_PARENT_DIR.resolve()
repo_root = find_repo_root_from_model_dir(MODEL_PARENT_DIR)
LABEL_DIR_TRAIN_THRESH, LABEL_DIR_TEST = get_label_dirs(repo_root)

PRED_SOURCE_TRAIN, PRED_SOURCE_TEST = get_pred_dirs(MODEL_PARENT_DIR)

print("Model parent:", MODEL_PARENT_DIR)
print("Repo root:", repo_root)
print("Label TRAIN_THRESH:", LABEL_DIR_TRAIN_THRESH)
print("Label TEST:", LABEL_DIR_TEST)
print("Pred source TRAIN:", PRED_SOURCE_TRAIN)
print("Pred source TEST:", PRED_SOURCE_TEST)

# Outputs (never overwrite originals)
THRESH_SWEEP_CSV   = MODEL_PARENT_DIR / "thresh_sweep_results.csv"
UTILITY_PLOT_PNG   = MODEL_PARENT_DIR / "utility_vs_threshold.png"
FINAL_METRICS_JSON = MODEL_PARENT_DIR / "final_metrics_official.json"
FINAL_TRAIN_DIR    = MODEL_PARENT_DIR / "pred_psv_TRAIN_THRESH_THRESHED"
FINAL_TEST_DIR     = MODEL_PARENT_DIR / "pred_psv_TEST_THRESHED"

# Temp dir for evaluator per-threshold files (safe)
TMP_PRED_DIR = MODEL_PARENT_DIR / "_tmp_pred_sweep"
TMP_PRED_DIR.mkdir(parents=True, exist_ok=True)

evaluator = load_official_evaluator(repo_root)

print("Running TWO-STAGE threshold sweep on TRAIN_THRESH (no cache)...")
df_sweep = run_threshold_sweep_two_stage(
    evaluator=evaluator,
    label_dir_train_thresh=LABEL_DIR_TRAIN_THRESH,
    pred_source_train=PRED_SOURCE_TRAIN,
    tmp_pred_dir=TMP_PRED_DIR,
    coarse_step=COARSE_STEP,
    refine_half_width=REFINE_HALF_WIDTH,
    refine_step=REFINE_STEP,
)

best_t = pick_best_threshold(df_sweep)
print("Best threshold by utility:", best_t)

# Save artifacts
df_sweep.to_csv(THRESH_SWEEP_CSV, index=False)
plot_utility(df_sweep, UTILITY_PLOT_PNG)
print("Saved sweep:", THRESH_SWEEP_CSV)
print("Saved plot:", UTILITY_PLOT_PNG)

# Write final thresholded outputs (SAFE folders)
materialize_predictions_from_source(PRED_SOURCE_TRAIN, FINAL_TRAIN_DIR, best_t)
print("Wrote final TRAIN folder:", FINAL_TRAIN_DIR)

if PRED_SOURCE_TEST is not None:
    materialize_predictions_from_source(PRED_SOURCE_TEST, FINAL_TEST_DIR, best_t)
    print("Wrote final TEST folder:", FINAL_TEST_DIR)
else:
    print("Skipped final TEST folder (no TEST predictions available).")

# Official metrics after threshold selection (TRAIN_THRESH)
auroc, auprc, acc, f1, util = official_eval(evaluator, LABEL_DIR_TRAIN_THRESH, FINAL_TRAIN_DIR)
final = {
    "model_parent_dir": str(MODEL_PARENT_DIR),
    "best_threshold": best_t,
    "train_thresh": {"auroc": auroc, "auprc": auprc, "accuracy": acc, "f_measure": f1, "utility": util},
}

# Optional TEST official eval (only if TEST labels exist AND test preds exist)
if (PRED_SOURCE_TEST is not None) and label_dir_has_sepsislabel(LABEL_DIR_TEST):
    auroc, auprc, acc, f1, util = official_eval(evaluator, LABEL_DIR_TEST, FINAL_TEST_DIR)
    final["test"] = {"auroc": auroc, "auprc": auprc, "accuracy": acc, "f_measure": f1, "utility": util}
else:
    final["test"] = {"note": "Skipped TEST official evaluation (missing SepsisLabel or missing TEST predictions)."}

FINAL_METRRICS_DIR = FINAL_METRICS_JSON.parent
FINAL_METRICS_JSON.write_text(json.dumps(final, indent=2))
print("Saved metrics:", FINAL_METRICS_JSON)

# Cleanup temp sweep dir (optional)
shutil.rmtree(TMP_PRED_DIR, ignore_errors=True)

final


Model parent: /teamspace/studios/this_studio/detecting_Sepsis/3_Model/Time-Series-Library/results/classification_Sepsis_TFT_HIGH_Mean_Impute_TemporalFusionTransformer_SepsisCSV_ftM_sl48_ll48_pl0_dm64_nh4_el2_dl1_df256_expand2_dc4_fc1_ebtimeF_dtTrue_TFT_on_CSV_High_Mean_Impute_0
Repo root: /teamspace/studios/this_studio/detecting_Sepsis
Label TRAIN_THRESH: /teamspace/studios/this_studio/detecting_Sepsis/data/raw_PSV/PSV_Patients_THRESH
Label TEST: /teamspace/studios/this_studio/detecting_Sepsis/data/raw_PSV/PSV_Patients_TEST
Pred source TRAIN: /teamspace/studios/this_studio/detecting_Sepsis/3_Model/Time-Series-Library/results/classification_Sepsis_TFT_HIGH_Mean_Impute_TemporalFusionTransformer_SepsisCSV_ftM_sl48_ll48_pl0_dm64_nh4_el2_dl1_df256_expand2_dc4_fc1_ebtimeF_dtTrue_TFT_on_CSV_High_Mean_Impute_0/pred_psv_TRAIN_THRESH_WORK
Pred source TEST: /teamspace/studios/this_studio/detecting_Sepsis/3_Model/Time-Series-Library/results/classification_Sepsis_TFT_HIGH_Mean_Impute_TemporalFusion

{'model_parent_dir': '/teamspace/studios/this_studio/detecting_Sepsis/3_Model/Time-Series-Library/results/classification_Sepsis_TFT_HIGH_Mean_Impute_TemporalFusionTransformer_SepsisCSV_ftM_sl48_ll48_pl0_dm64_nh4_el2_dl1_df256_expand2_dc4_fc1_ebtimeF_dtTrue_TFT_on_CSV_High_Mean_Impute_0',
 'best_threshold': 0.2615,
 'train_thresh': {'auroc': np.float64(0.8239374005123444),
  'auprc': np.float64(0.08735154399897739),
  'accuracy': 0.7779450261780104,
  'f_measure': 0.10604663417204585,
  'utility': np.float64(0.383405760182216)},
 'test': {'auroc': np.float64(0.8402254302517654),
  'auprc': np.float64(0.12145947896272234),
  'accuracy': 0.7584618749276351,
  'f_measure': 0.10315507893668346,
  'utility': np.float64(0.39365348299535835)}}